#Ingest and load python jupyter notebook script

1. Imports
2. Currency pairs data
3. Trading dates (from, to)
4. OHLC trades data. 

In [ ]:
##1.Imports

In [120]:
import requests
import json
import pandas as pd
import datetime
from datetime import date
from datetime import datetime as dt

In [ ]:
##2. Currency pairs data

In [109]:
tradableassetpairs = 'https://api.kraken.com/0/public/AssetPairs'
response_pairs = requests.get(tradableassetpairs)
data = response_pairs.text
json_pairs = json.loads(data)
pairs = json_pairs['result']

#print(pairs)

#array_pairs = pd.DataFrame(columns=('altname','wsname','indexofslash','from','to','quote'))

i = 0

for pair in pairs.values():
    indexofslash = pair.get('wsname').index('/')
    array_pairs.loc[i] = pair.get('altname'),pair.get('wsname'),indexofslash,pair.get('wsname')[:indexofslash],pair.get('wsname')[indexofslash+1:],pair.get('quote')
    i = i + 1


#Row count
print('Row counts: '+len(array_pairs.index))

#Filter array_pairs by quote (ZUSD, ZEUR, ZAUD, ZGBP)
quotefilter = 'ZAUD'
array_pairs = array_pairs.loc[array_pairs['quote'] == quotefilter]

#Print DataFrame
print(array_pairs)

#Save array_pairs dataframe into csv
array_pairs.to_csv('currency_pairs.csv',index=False)
    

     altname    wsname  indexofslash  from   to quote
2    AAVEAUD  AAVE/AUD             4  AAVE  AUD  ZAUD
12    ADAAUD   ADA/AUD             3   ADA  AUD  ZAUD
56   ATOMAUD  ATOM/AUD             4  ATOM  AUD  ZAUD
83    BCHAUD   BCH/AUD             3   BCH  AUD  ZAUD
147   DOTAUD   DOT/AUD             3   DOT  AUD  ZAUD
173   ETHAUD   ETH/AUD             3   ETH  AUD  ZAUD
181   EURAUD   EUR/AUD             3   EUR  AUD  ZAUD
196   FILAUD   FIL/AUD             3   FIL  AUD  ZAUD
231   GRTAUD   GRT/AUD             3   GRT  AUD  ZAUD
283   KSMAUD   KSM/AUD             3   KSM  AUD  ZAUD
294  LINKAUD  LINK/AUD             4  LINK  AUD  ZAUD
312   LTCAUD   LTC/AUD             3   LTC  AUD  ZAUD
464   SNXAUD   SNX/AUD             3   SNX  AUD  ZAUD
523  USDCAUD  USDC/AUD             4  USDC  AUD  ZAUD
529  USDTAUD  USDT/AUD             4  USDT  AUD  ZAUD
550   XBTAUD   XBT/AUD             3   XBT  AUD  ZAUD
582   XRPAUD   XRP/AUD             3   XRP  AUD  ZAUD
588   XTZAUD   XTZ/AUD      

In [ ]:
##3.Trading dates (from, to)

In [128]:
#current date
start_date = str(date.today())
#Set days back for retrieving trades data
days_minus = 7 #a week

date_formated = datetime.datetime.strptime(start_date, "%Y-%m-%d")

#get days ago date_formated
end_date = date_formated - datetime.timedelta(days=days_minus)

#print end_date unix timestamp
#print(end_date.timestamp())
end_date_unixtimestamp = end_date.timestamp()

print('Date From: '+str(end_date)+', Date To: '+str(start_date))

Date From: 2022-09-15 00:00:00,Date To: 2022-09-22


In [ ]:
##4. OHLC trades data.

In [129]:
ohlcurl = 'https://api.kraken.com/0/public/OHLC?'
unixtime = str(end_date_unixtimestamp)
interval = str(1440)
concatohlcurl = ''

dataframe_ohlc = pd.DataFrame(columns=('from','to','altname','unixtimestamp','date','open','high','low','close','vwap','volume','count'))

j = 0

for altname in array_pairs['altname']:
    
    concatohlcurl = ohlcurl +'interval=' + interval + '&since=' + unixtime + '&pair=' + altname
    
    #print(altname+'-'+concatohlcurl)

    
    #print(concatohlcurl)
    
    ohlc = requests.get(concatohlcurl)
    tradesdata = ohlc.text
    json_trades = json.loads(tradesdata)
        
    #print(json_trades)
    
    try:
        trades = json_trades['result'][altname]
    except:
        trades = ''
    
    #print(trades)
    
    for trade in trades:
        
        #print(trade)
        
        try:
            pairfrom = array_pairs['from'].loc[array_pairs['altname']==altname].iloc[0]
        except:
            pairfrom = ''
        
        try:
            pairto = array_pairs['to'].loc[array_pairs['altname']==altname].iloc[0]
        except:
            pairto = ''
        
        unixtimestamp = str(trade[0])
        convertedunixtimestampintodate = dt.utcfromtimestamp(trade[0]).strftime('%Y-%m-%d')
        open = trade[1]
        high = trade[2]
        low = trade[3]
        close = trade[4]
        vwap = trade[5]
        volume = trade[6]
        count = str(trade[7])
        
        #print(pairfrom+'-'+pairto+'-'+altname+'-'+unixtimestamp+'-'+convertedunixtimestampintodate+'-'+open+'-'+high+'-'+low+'-'+close+'-'+vwap+'-'+volume+'-'+count)
        dataframe_ohlc.loc[j] = pairfrom,pairto,altname,unixtimestamp,convertedunixtimestampintodate,open,high,low,close,vwap,volume,count
        j = j + 1        

print(dataframe_ohlc)

#save dataframe_ohlc to csv
dataframe_ohlc.to_csv('ohlctrades.csv',index=False)

     from   to  altname unixtimestamp        date    open    high     low  \
0    AAVE  AUD  AAVEAUD    1663200000  2022-09-15  123.46  123.46  121.12   
1    AAVE  AUD  AAVEAUD    1663286400  2022-09-16  117.89  119.12  116.41   
2    AAVE  AUD  AAVEAUD    1663372800  2022-09-17  119.41  120.80  118.35   
3    AAVE  AUD  AAVEAUD    1663459200  2022-09-18  120.55  121.19  106.99   
4    AAVE  AUD  AAVEAUD    1663545600  2022-09-19  108.66  114.78  106.32   
..    ...  ...      ...           ...         ...     ...     ...     ...   
147   YFI  AUD   YFIAUD    1663459200  2022-09-18   12340   12340   12211   
148   YFI  AUD   YFIAUD    1663545600  2022-09-19   12211   12211   12211   
149   YFI  AUD   YFIAUD    1663632000  2022-09-20   12211   12211   12211   
150   YFI  AUD   YFIAUD    1663718400  2022-09-21   12211   12211   12211   
151   YFI  AUD   YFIAUD    1663804800  2022-09-22   12211   12211   12211   

      close    vwap      volume count  
0    121.12  121.49  2.29695286    